In [1]:
import pandas as pd

feature_name_df = pd.read_csv('./datasets/human_activity/features.txt',sep='\s+',
           header=None,names=['column_index','column_name'])
feature_name = feature_name_df.iloc[:,1].values.tolist()

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name'] =new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] >0 else x[0],axis=1)
    
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

def get_human_dataset():
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('./datasets/human_activity/features.txt',sep='\s+',
           header=None,names=['column_index','column_name'])
    # 중복된 피처명을 수정하는 get_new_feature_name_df를 이용 신규 피쳐명 df생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DF에 피쳐명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터 셋을 DF로 로딩 컬러명은 feature_name적용
    x_train = pd.read_csv('./datasets/human_activity/train/X_train.txt',sep="\s+",names=feature_name)
    x_test = pd.read_csv('./datasets/human_activity/test/X_test.txt',sep="\s+",names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터를 DF로 할당 컬럼명은 action으로 부여
    y_train = pd.read_csv('./datasets/human_activity/train/y_train.txt',sep="\s+",header=None,names=['action'])
    y_test = pd.read_csv('./datasets/human_activity/test/y_test.txt',sep="\s+",header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 변환
    return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test=get_human_dataset()

#### sklearn.ensemble.GradientBoostingClassifier

* class sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

* parameters:
    - loss : {‘log_loss’, ‘deviance’, ‘exponential’}, default=’log_loss',손실함수로 보통 diviance 잔차의 경사하강법을 이용하여 사용함 
    - learning_rate :float, default=0.1 (0~1사이 값) 학습을 진행할 때마다 적용하는 학습률 너무작은값을 적용하면 업데이트되는 값이 작아져서 최소 오류값을 찾아 예측 성능이 높아질 수 있다 또한 너무 작은값을 하면 시간이오래걸림 반면 너무 큰값을 사용시 과적합염려
    - n_estimators
    - subsample : 기본값은 1이며 전체학습데이터를 기반으로 학습 과적합이 염려되면 1보다 작은값 사용
    - max_depth
    - min_samples_split
    - max_samples_leaf
    - max_features : default = 'None'
    - max_leaf_nodes
    - criterion : {‘friedman_mse’, ‘squared_error’}, default=’friedman_mse’

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
gb_clf = GradientBoostingClassifier(random_state=156)
gb_clf.fit(x_train,y_train)
gb_pred = gb_clf.predict(x_test)
gb_accuracy = accuracy_score(y_test,gb_pred)
print(gb_accuracy)

c:\Users\msi\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9389209365456397


#### sklearn.ensemble.AdaBoostClassifier
* class sklearn.ensemble.AdaBoostClassifier(estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None, base_estimator='deprecated')

* parameters
    - estimator : object, default=None
      The base estimator from which the boosted ensemble is built. Support for sample weighting is required, as well as proper classes_ and n_classes_ attributes. If None, then the base estimator is DecisionTreeClassifier initialized with max_depth=1.

.
   - n_estimators :int, default=50 : The maximum number of estimators at which boosting is terminated. In case of perfect fit, the learning procedure is stopped early. Values must be in the range [1, inf).

learning_rate : float, default=1.0
Weight applied to each classifier at each boosting iteration. A higher learning rate increases the contribution of each classifier. There is a trade-off between the learning_rate and n_estimators parameters. Values must be in the range (0.0, inf).

algorithm : {‘SAMME’, ‘SAMME.R’}, default=’SAMME.R’
If ‘SAMME.R’ then use the SAMME.R real boosting algorithm. estimator must support calculation of class probabilities. If ‘SAMME’ then use the SAMME discrete boosting algorithm. The SAMME.R algorithm typically converges faster than SAMME, achieving a lower test error with fewer boosting iterations.

random_state : int, RandomState instance or None, default=None
Controls the random seed given at each estimator at each boosting iteration. Thus, it is only used when estimator exposes a random_state. Pass an int for reproducible output across multiple function calls. See Glossary.

base_estimator : object, default=None
The base estimator from which the boosted ensemble is built. Support for sample weighting is required, as well as proper classes_ and n_classes_ attributes. If None, then the base estimator is DecisionTreeClassifier initialized with max_depth=1.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
ada_clf = AdaBoostClassifier(random_state=156)
ada_clf.fit(x_train,y_train)
ada_pred = ada_clf.predict(x_test)
ada_accuracy = accuracy_score(y_test,ada_pred)
print(ada_accuracy)

#### xgboost

- XGBoost는 다수의 하이퍼파라미터가 존재하며, 다음 세가지 범주로 나뉜다.

- 일반 파라미터
 부스팅을 수행할 때 트리를 사용할지, 선형 모델을 사용할지 등을 고른다.
- 부스터 파라미터
 선택한 부스터에 따라서 적용할 수 있는 파라미터 종류가 다르다.
- 학습 과정 파라미터
 학습 시나리오를 결정한다.

- 일반 파라미터

- booster [기본값 = gbtree]
어떤 부스터 구조를 쓸지 결정한다.
의사결정기반모형(gbtree), 선형모형(gblinear), dart가 있다.
- n_jobs
 XGBoost를 실행하는 데 사용되는 병렬 스레드 수
- verbosity [기본값 = 1]
유효한 값은 0 (무음), 1 (경고), 2 (정보), 3 (디버그)

- 부스터 파라미터

- gbtree Booster의 파라미터
learning_rate [ 기본값 : 0.3 ]
learning rate이다.
learning rate가 높을수록 과적합 하기 쉽다.
- n_estimators [ 기본값 : 100 ]
생성할 weak learner의 수
learning_rate가 낮을 땐, n_estimators를 높여야 과적합이 방지된다.
- max_depth [ 기본값 : 6 ]
트리의 maximum depth이다.
적절한 값이 제시되어야 하고 보통 3-10 사이 값이 적용된다.
max_depth가 높을수록 모델의 복잡도가 커져 과적합 하기 쉽다.
- min_child_weight [ 기본값 : 1 ]
관측치에 대한 가중치 합의 최소를 말한다.
값이 높을수록 과적합이 방지된다.
- gamma [ 기본값 : 0 ]
리프노드의 추가분할을 결정할 최소손실 감소값이다.
해당값보다 손실이 크게 감소할 때 분리한다.
값이 높을수록 과적합이 방지된다.
- subsample [ 기본값 : 1 ]
weak learner가 학습에 사용하는 데이터 샘플링 비율이다.
보통 0.5 ~ 1 사용된다.
값이 낮을수록 과적합이 방지된다.
- colsample_bytree [ 기본값 : 1 ]
각 tree 별 사용된 feature의 퍼센테이지이다.
보통 0.5 ~ 1 사용된다.
값이 낮을수록 과적합이 방지된다.
- lambda [기본값 = 1, 별칭 : reg_lambda]
가중치에 대한 L2 Regularization 적용 값
피처 개수가 많을 때 적용을 검토
이 값이 클수록 과적합 감소 효과
- alpha [기본값 = 0, 별칭 : reg_alpha]
가중치에 대한 L1 Regularization 적용 값
피처 개수가 많을 때 적용을 검토
이 값이 클수록 과적합 감소 효과

- 학습 과정 파라미터

- objective [ 기본값 : reg = squarederror ]
- reg : squarederror
제곱 손실이 있는 회귀
- binary : logistic (binary-logistic classification)
이항 분류 문제 로지스틱 회귀 모형으로 반환값이 클래스가 아니라 예측 확률
- multi : softmax
다항 분류 문제의 경우 소프트맥스(Softmax)를 사용해서 분류하는데 반횐되는 값이 예측확률이 아니라 클래스임. 또한 num_class도 지정해야함.
- multi : softprob
각 클래스 범주에 속하는 예측확률을 반환함.
- count : poisson (count data poison regression) 등 다양하다.
- eval_metric : 모델의 평가 함수를 조정하는 함수다.설정한 objective 별로 기본설정값이 지정되어 있다.
    - rmse: root mean square error
    - mae: mean absolute error
    - logloss: negative log-likelihood
    - error: Binary classification error rate (0.5 threshold)
    - merror: Multiclass classification error rate
    - mlogloss: Multiclass logloss
    - auc: Area under the curve
    - map (mean average precision)등, 해당 데이터의 특성에 맞게 평가 함수를 조정한다.
- seed [ 기본값 : 0 ] :재현가능하도록 난수를 고정시킴.

In [20]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
y_train = y_train-1
y_test = y_test-1
# 모델 선언
xgb_clf = xgb.XGBClassifier()

# 모델 훈련
xgb_clf.fit(x_train,y_train) 

# 모델 예측
xgb_pred = xgb_clf.predict(x_test)
xgb_accuracy = accuracy_score(y_test,xgb_pred)
print(xgb_accuracy)

0.9389209365456397
